# Travel Plan Team

## Prepare models

In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.models import UserMessage
from dotenv import load_dotenv
import os

load_dotenv() # Load environment variables from .env file

az_model_client = AzureOpenAIChatCompletionClient(
    model = "gpt-4o-2024-11-20",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_URL"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment = "gpt-4o",
    api_version = "2024-12-01-preview"
)

az_o3_model_client = AzureOpenAIChatCompletionClient(
    model = "o3-mini",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_URL"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment = "o3-mini",
    api_version = "2024-12-01-preview"
)

## Prepare Agents

In [2]:
from autogen_agentchat.agents import AssistantAgent
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=az_o3_model_client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=az_model_client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=az_model_client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=az_o3_model_client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

## Organize team

In [3]:
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)
await Console(group_chat.run_stream(task="Plan a 3 day trip to Nich, Franch."))

---------- TextMessage (user) ----------
Plan a 3 day trip to Nich, Franch.
---------- TextMessage (planner_agent) ----------
Below is an example 3‑day itinerary for a trip to Nice, France. (If you meant a different destination in France, please let me know and I can adjust the plan accordingly!)

──────────────────────────────
Day 1: Discover Old Town & Seafront

Morning
• Arrive in Nice and check into your centrally located hotel (consider areas like the Old Town/Vieux Nice or near the Promenade des Anglais for easy access).
• Start your day with a stroll through the famous Cours Saleya market. In the morning the market bursts with fresh produce, local cheeses, and flowers.
• Enjoy a light breakfast at one of the cafés in the market.

Midday
• Wander through the narrow, colorful streets of Vieux Nice (Old Town). Admire its Baroque architecture, visit the Saint Réparable Cathedral, and soak in the lively atmosphere.
• Find a local bistro for lunch where you can sample Niçoise specialt

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Plan a 3 day trip to Nich, Franch.', type='TextMessage'), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=2117), metadata={}, content='Below is an example 3‑day itinerary for a trip to Nice, France. (If you meant a different destination in France, please let me know and I can adjust the plan accordingly!)\n\n──────────────────────────────\nDay 1: Discover Old Town & Seafront\n\nMorning\n• Arrive in Nice and check into your centrally located hotel (consider areas like the Old Town/Vieux Nice or near the Promenade des Anglais for easy access).\n• Start your day with a stroll through the famous Cours Saleya market. In the morning the market bursts with fresh produce, local cheeses, and flowers.\n• Enjoy a light breakfast at one of the cafés in the market.\n\nMidday\n• Wander through the narrow, colorful streets of Vieux Nice (Old Town). Admire its Baro